<a href="https://colab.research.google.com/github/suajeong52/CNN/blob/main/%EC%88%98%EC%95%84Resnet(%EC%A0%95%EA%B7%9C%ED%99%94%2Brelu).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
NUM_CLASSES = 10

x_train = x_train.astype('float32')/255.
x_test  = x_test.astype('float32')/255.

y_train = to_categorical(y_train)
y_test  = to_categorical(y_test)

170508288/170498071 [==============================] - 11s 0us/step


In [ ]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-fherk0fa
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-fherk0fa
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101077 sha256=0ab7500c9df2eede4885fb5cd401974e010ddcf39c8734b55e68ec8b27f061e4
  Stored in directory: /tmp/pip-ephem-wheel-cache-85_3jfm6/wheels/bb/1f/f2/b57495012683b6b20bbae94a3915ec79753111452d79886abc
Successfully built keras-contrib


In [ ]:
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate, Layer, InputSpec, Activation, LeakyReLU
from tensorflow.keras.activations import sigmoid 
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError 
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from tensorflow.keras.layers import UpSampling2D, Conv2D, Conv2DTranspose
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.layers import add

from tensorflow.keras.callbacks import Callback, LearningRateScheduler
from tensorflow.keras.callbacks import ModelCheckpoint
import os

In [ ]:
class CustomCallback(Callback):
    
    def __init__(self, run_folder, print_every_n_batches, initial_epoch, vae, ):
        self.epoch = initial_epoch
        self.run_folder = run_folder
        self.print_every_n_batches = print_every_n_batches
        self.vae = vae

    def on_batch_end(self, batch, logs={}):  
      a=0
    #  if batch % self.print_every_n_batches == 0:
    #    print(batch)
     #     self.model.predict()
     #     potential_by_cnn =  model.predict(X_test)


     #       z_new = np.random.normal(size = (1,self.vae.z_dim))
     #       reconst = self.vae.decoder.predict(np.array(z_new))[0].squeeze()

      #      filepath = os.path.join(self.run_folder, 'images', 'img_' + str(self.epoch).zfill(3) + '_' + str(batch) + '.jpg')
      #      if len(reconst.shape) == 2:
      #          plt.imsave(filepath, reconst, cmap='gray_r')
      #      else:
      #          plt.imsave(filepath, reconst)

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch += 1



def step_decay_schedule(initial_lr, decay_factor=0.5, step_size=1):
    '''
    Wrapper function to create a LearningRateScheduler with step decay schedule.
    '''
    def schedule(epoch):
        new_lr = initial_lr * (decay_factor ** np.floor(epoch/step_size))
        
        return new_lr

    return LearningRateScheduler(schedule)

In [ ]:
weight_init = RandomNormal(mean=0., stddev=0.02)

def conv7s1(layer_input, filters, final):
  y = ReflectionPadding2D(padding =(3,3))(layer_input)
  y = Conv2D(filters, kernel_size=(7,7), strides=1, padding='valid', kernel_initializer = weight_init)(y)
  if final:
    y = sigmoid(y) 
  else: 
    y = LeakyReLU()(y)
    y = InstanceNormalization(axis = -1, center = False, scale = False)(y)

  return y

def downsample(layer_input,filters):
  y = Conv2D(filters, kernel_size=(3,3), strides=2, padding='same', kernel_initializer = weight_init)(layer_input)
  y = LeakyReLU()(y)
  y = InstanceNormalization(axis = -1, center = False, scale = False)(y)
  return y

def residual(layer_input, filters):
  shortcut = layer_input
  y = ReflectionPadding2D(padding =(1,1))(layer_input)
  y = Conv2D(filters, kernel_size=(3, 3), strides=1, padding='valid', kernel_initializer = weight_init)(y)
  y = LeakyReLU()(y)
  y = InstanceNormalization(axis = -1, center = False, scale = False)(y)
  return add([shortcut, y])

def upsample(layer_input,filters):
  y = Conv2DTranspose(filters, kernel_size=(3, 3), strides=2, padding='same', kernel_initializer = weight_init)(layer_input)
  y = LeakyReLU()(y)
  y = InstanceNormalization(axis = -1, center = False, scale = False)(y)
  return y




In [ ]:
class ReflectionPadding2D(Layer):
    def __init__(self, padding=(1, 1), **kwargs):
        self.padding = tuple(padding)
        self.input_spec = [InputSpec(ndim=4)]
        super(ReflectionPadding2D, self).__init__(**kwargs)

    def compute_output_shape(self, s):
        """ If you are using "channels_last" configuration"""
        return (s[0], s[1] + 2 * self.padding[0], s[2] + 2 * self.padding[1], s[3])

    def call(self, x, mask=None):
        w_pad,h_pad = self.padding
        return tf.pad(x, [[0,0], [h_pad,h_pad], [w_pad,w_pad], [0,0] ], 'REFLECT')

In [ ]:
gen_n_filters = 8

img = Input(shape=(32,32,3))

y = conv7s1(img, gen_n_filters, False)


y = downsample(y, gen_n_filters * 2)
for i in range(10):
  y = residual(y, gen_n_filters * 2)


y = downsample(y, gen_n_filters * 2)
for i in range(10):
  y = residual(y, gen_n_filters * 2)

y = Flatten()(y)

y = Dense(128, activation ='relu')(y)
y = LeakyReLU()(y)

output = Dense(10, activation = 'softmax')(y)

   
model = Model(img, output)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 reflection_padding2d (Reflecti  (None, 38, 38, 3)   0           ['input_1[0][0]']                
 onPadding2D)                                                                                     
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 8)    1184        ['reflection_padding2d[0][0]']   
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 32, 32, 8)    0           ['conv2d[0][0]']             

In [ ]:
opt = Adam(learning_rate=0.001)
model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics= ['accuracy'])

In [ ]:
model.fit(x_train, y_train, batch_size = 32, epochs = 30, shuffle=True, verbose=1)

Epoch 1/30
1563/1563 [==============================] - 77s 36ms/step - loss: 1.9192 - accuracy: 0.3373
Epoch 2/30
1563/1563 [==============================] - 57s 36ms/step - loss: 1.5028 - accuracy: 0.4655
Epoch 3/30
1563/1563 [==============================] - 58s 37ms/step - loss: 1.3073 - accuracy: 0.5380
Epoch 4/30
1563/1563 [==============================] - 56s 36ms/step - loss: 1.1535 - accuracy: 0.5940
Epoch 5/30
1563/1563 [==============================] - 56s 36ms/step - loss: 1.0248 - accuracy: 0.6444
Epoch 6/30
1563/1563 [==============================] - 56s 36ms/step - loss: 0.9072 - accuracy: 0.6830
Epoch 7/30
1563/1563 [==============================] - 56s 36ms/step - loss: 0.8039 - accuracy: 0.7198
Epoch 8/30
1563/1563 [==============================] - 56s 36ms/step - loss: 0.7082 - accuracy: 0.7530
Epoch 9/30
1563/1563 [==============================] - 55s 35ms/step - loss: 0.6173 - accuracy: 0.7834
Epoch 10/30
1563/1563 [==============================] - 55s 35m

In [ ]:
model.evaluate(x_test, y_test, batch_size=1000)

10/10 [==============================] - 2s 91ms/step - loss: 3.1196 - accuracy: 0.6074


[3.119581937789917, 0.6074000000953674]